In [62]:
import os
import json
import csv

def count_characters_in_string(input_string):
    letter_count = 0
    for char in input_string:
        if char.isalpha():
            letter_count += 1

    return letter_count

def extract_values(directory, person):
    audience, example = os.path.split(directory)[1].split("_", 1)
    
    file_path = str(directory)+"/snapshot_final_assistant_with_scores.json"
    #print(file_path)
    
    with open(file_path, 'r') as file:
        data = json.load(file)
        
        extracted_values = []
        
        contents = []
        rouge_scores = []
        bleu_scores = []
        bleurt_scores = []
        ter_scores = []
        
        
        for obj in data:
            #print(obj)
            for keys, values in obj.items():
                #print(keys, values)
                
                # Getting all the occurences of values
                if keys == 'content':
                    contents.append(values)
                if keys == 'rougeL':
                    rouge_scores.append(values)
                if keys == 'bleu':
                    bleu_scores.append(values)
                if keys == 'scores':
                    bleurt_scores.append(values[0])
                if keys == 'score':
                    ter_scores.append(values)
        
        #print(contents)
        if len(rouge_scores) == 1: # To manage the None_none category adding 0,0 such that it stays formatted as (manual,base,add)
            rouge_scores.append(0)
            rouge_scores.append(0)
            
            bleu_scores.append(0)
            bleu_scores.append(0)
            
            bleurt_scores.append(0)
            bleurt_scores.append(0)
            
            ter_scores.append(0)
            ter_scores.append(0)
        
        
        final_summary = contents[-1].replace("\n","")
        word_count = len(contents[-1].split(" "))
        character_count = count_characters_in_string(contents[-1])
        
        #print(final_summary)
        #print("word count =",word_count)
        #print("character_count =",character_count)
        #print("rouge_scores =",rouge_scores)
        #print("bleu_scores =",bleu_scores)
        #print("bleurt_scores =",bleurt_scores)
        #print("ter_scores =",ter_scores)
                
        #extracted_values = [final_summary, audience, example, person, "ground_truth", word_count, character_count, rouge_scores, bleu_scores, bleurt_scores, ter_scores]
        
        if person == '1_250':
            person = 1
        
        # Flattening out the values
        extracted_values = [
            [final_summary, audience, example, person, "manual", word_count, character_count, rouge_scores[0], bleu_scores[0], bleurt_scores[0], ter_scores[0]],
            [final_summary, audience, example, person, "baseline", word_count, character_count, rouge_scores[1], bleu_scores[1], bleurt_scores[1], ter_scores[1]],
            [final_summary, audience, example, person, "additional", word_count, character_count, rouge_scores[2], bleu_scores[2], bleurt_scores[2], ter_scores[2]]
        ]
        
        return extracted_values

        
def process_persons_directories(directories):
    output_rows =[]
    for directory in directories:
        extracted_values = extract_values(directory, person)
        for ground_truths in extracted_values:
            output_rows.append(ground_truths)
    return output_rows


#def write_to_csv(file_path, rows):
    #headers = ['person', 'final_summary', 'word_count', 'character_count', 'ground_truth', 'rouge', 'bleu', 'bleurt', 'ter']
    #with open(file_path, 'w', newline='') as file:
        #writer = csv.writer(file)
        #writer.writerow(headers)
        #writer.writerows(rows)
    
    #print("CSV file created successfully.")

In [73]:
person = 3
#for person in persons:
    
directories = [
                "../prompt_engineering/p"+str(person)+"/none_none", "../prompt_engineering/p"+str(person)+"/none_manual_example", "../prompt_engineering/p"+str(person)+"/none_masked_manual_example", "../prompt_engineering/p"+str(person)+"/none_masked_template",
                "../prompt_engineering/p"+str(person)+"/self_none", "../prompt_engineering/p"+str(person)+"/self_manual_example", "../prompt_engineering/p"+str(person)+"/self_masked_manual_example", "../prompt_engineering/p"+str(person)+"/self_masked_template",
                "../prompt_engineering/p"+str(person)+"/peer_none", "../prompt_engineering/p"+str(person)+"/peer_manual_example", "../prompt_engineering/p"+str(person)+"/peer_masked_manual_example", "../prompt_engineering/p"+str(person)+"/peer_masked_template",
                "../prompt_engineering/p"+str(person)+"/manager_none", "../prompt_engineering/p"+str(person)+"/manager_manual_example", "../prompt_engineering/p"+str(person)+"/manager_masked_manual_example", "../prompt_engineering/p"+str(person)+"/manager_masked_template"
              ]     
    
output_rows = process_persons_directories(directories)

In [74]:
len(output_rows)

48

In [75]:
import pandas as pd
import numpy as np
from itertools import chain

column_headers = ['final_summary', 'audience', 'example', 'person', 'ground_truth', 'word_count', 'character_count', 'rouge', 'bleu', 'bleurt', 'ter']
# The extra additions were to accomodate the flattening of the values based on ground truths
row_headers = [
                ["none_none"]*3, ["none_manual"]*3, ["none_masked_manual"]*3, ["none_masked_template"]*3,
                ["self_none"]*3, ["self_manual"]*3, ["self_masked_manual"]*3, ["self_masked_template"]*3,
                ["peer_none"]*3, ["peer_manual"]*3, ["peer_masked_manual"]*3, ["peer_masked_template"]*3,
                ["manager_none"]*3, ["manager_manual"]*3, ["manager_masked_manual"]*3, ["manager_masked_template"]*3
              ]
row_headers_flatted = list(chain.from_iterable(row_headers))

df = pd.DataFrame(output_rows, columns = column_headers)

In [76]:
df

,final_summary,audience,example,person,ground_truth,word_count,character_count,rouge,bleu,bleurt,ter
0,"In this investigation, the user searched for a...",none,none,3,manual,312,1720,0.088496,0.000000,-0.929089,93.333333
1,"In this investigation, the user searched for a...",none,none,3,baseline,312,1720,0.000000,0.000000,0.000000,0.000000
2,"In this investigation, the user searched for a...",none,none,3,additional,312,1720,0.000000,0.000000,0.000000,0.000000
3,This session focused on investigating a comple...,none,manual_example,3,manual,135,716,0.272727,0.151788,-0.127079,86.666667
4,This session focused on investigating a comple...,none,manual_example,3,baseline,135,716,0.183406,0.072728,-0.475221,205.185185
5,This session focused on investigating a comple...,none,manual_example,3,additional,135,716,0.193370,0.047168,-0.336326,145.185185
6,This investigation session focused on arms dea...,none,masked_manual_example,3,manual,186,993,0.151420,0.000000,-0.655520,90.860215
7,This investigation session focused on arms dea...,none,masked_manual_example,3,baseline,186,993,0.258317,0.159350,-0.091765,137.096774
8,This investigation session focused on arms dea...,none,masked_manual_example,3,additional,186,993,0.313253,0.257360,-0.068301,100.537634
9,They focused on multiple topics in this analys...,none,masked_template,3,manual,342,1880,0.116910,0.000000,-0.912865,93.895349


In [77]:
file_path = '../prompt_engineering/p3.csv'
df.to_csv(file_path, index=False)